#  4. Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


In [2]:
data = pd.read_csv('retail_train.csv')


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
from metrics import precision_at_k, recall_at_k, hit_rate_at_k_mean_v2, precision_at_k_mean, recall_at_k_mean
from utilis import prefilter_items
from recomenders3 import MainRecommender
#from recomender2 import MainRecommender as Mn

In [4]:
data_train = prefilter_items(data_train)
data_train.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [5]:
rec = MainRecommender(data_train)

C:\Users\user\Documents\anaconda3\Lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [6]:
top_purchase = rec.overall_top_purchases
top_purchase[:5]

[1082185, 6534178, 1029743, 995242, 1106523]

In [7]:
rec.get_als_recommendations(rec.id_to_userid[0])

[885290, 5569374, 995242, 1100972, 1005186]

In [11]:
rec.get_als_recommendations(9)

[1082185, 6534178, 1029743, 995242, 981760]

In [9]:
rec.get_similar_users_recommendation(9)

[1082185, 1082185, 6534178, 1029743, 995242]

In [ ]:
rec.fit(data_test)

In [13]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [ ]:
rec.id_to_userid[93]

In [ ]:
result['actual'][1]

In [15]:
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


In [14]:
precision_at_k(result['actual'][1], rec.get_als_recommendations(rec.id_to_userid[3]))

0.0

In [ ]:
rec.get_als_recommendations(rec.id_to_userid[3])

In [ ]:
als3 = []
for el in rec.userid_to_id.keys():
    als3.append(rec.get_als_recommendations(el))
    


In [ ]:
als3

In [ ]:
als3_pd = pd.DataFrame({'user_id': rec.userid_to_id.keys(), 'als3': als3})
als3_pd

In [ ]:
result=result.merge(als3_pd, on='user_id')

In [ ]:
result

In [ ]:
hit_rate_at_k_mean_v2(result['als3'], result['actual'], k=3)

In [ ]:
als = []
for el in list(range(1,2043)):
    als.append(rec.get_als_recommendations(el))
    
als

In [ ]:
als = []
for el in result['user_id']:
    als.append(rec.get_als_recommendations(el))
als

In [ ]:
user_id_for_pred = []
for el in result['user_id'][:-2]:
    user_id_for_pred.append(rec.id_to_userid[el])

In [ ]:
als2 = []
for el in user_id_for_pred:
    als2.append(rec.get_als_recommendations(el))
als2

In [ ]:
als2.append([0])
als2.append([0])
result['als2'] = als2

In [ ]:
result['als'] = als

In [ ]:
result

In [ ]:
hit_rate_at_k_mean_v2(result['als'], result['actual'], k=3)

In [ ]:
precision_at_k_mean(result['als'], result['actual'], k=3)

In [ ]:
precision_at_k_mean(result['als2'], result['actual'], k=3)